In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from matplotlib.lines import Line2D
import os
import json
import matplotlib.ticker as ticker
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from scipy.interpolate import make_interp_spline, BSpline
os.chdir("C:/Users/Zber/Documents/Dev_program/OpenRadar")
from FER.utils import MapRecord
from sklearn.model_selection import train_test_split

In [13]:
from sklearn.decomposition import PCA
res_path = "C:/Users/Zber/Documents/Dev_program/OpenRadar/FER/results"
pca = PCA(n_components=2)
kd_path = os.path.join(res_path, 'Supervision_SUM_image2D_KD_20220710-200042')
kd_dist = []
kd_smap= np.load(os.path.join(kd_path,'smap_{}.npy'.format(0)))
kd_tmap= np.load(os.path.join(kd_path,'tmap_{}.npy'.format(0)))
kd_tmap = kd_tmap.reshape((242, 30,512,7,7))
kd_tmap = np.apply_over_axes(np.mean, kd_tmap, [1,3,4])

In [ ]:
kd_tmap

In [15]:
np.squeeze(kd_tmap).shape

(242, 512)

In [9]:
kd_smap.shape

(242, 4096)

In [2]:
def annotation_update(record_list, width=100, total_frame=300):
    for record in record_list:
        # if record.num_frames < width:
        #     pad = (width - record.num_frames)//2
        #     if record.onset < pad:
        #         record.peak += pad*2

        #     elif (total_frame - record.peak) < pad:
        #         record.onset -= pad*2
        #     else:
        #         record.onset -= pad
        #         record.peak += pad
        # else:
        #     pad = record.num_frames - width
        #     record.peak -= pad

        record.path = record.path.replace("Raw_0.bin","{}.npy").replace("\\", "/")

        if record.num_frames != 100:
            record.peak += 1
        assert record.num_frames == 100, 'the num of frames must equal to 100!'
    return record_list


def annotation_attention(record_list, width=30):
    for record in record_list:
        record.onset = math.floor(record.onset * 3 / 10)
        record.peak = record.onset + width - 1
        record.path = record.relative_path.replace("_{}.npy","")
    return record_list


def annotation_append(subs=['S6','S7']):
    str_arr = []
    str_format = "{} {} {} {} {} {} {} {}"
    npy_path = "{}_{}"
    emotion = 'Neutral'
    # subs = ['S0', 'S1', 'S2', 'S3', 'S4', 'S5']

    for sub in subs:
        for i in range(0,30):
            path = (os.path.join(sub, npy_path.format(emotion, i,)) + '_{}.npy').replace("\\", "/")
            label = "0"
            onset = 31
            peak = 130
            offset = -1
            e1 = 0
            e2 = 0
            e3 = 0
            str_arr.append(str_format.format(path, label, onset, peak, offset, e1, e2 , e3))
    return str_arr


def data_split(record_list):
    labels = [r.label for r in record_list]
    train, test = train_test_split(record_list, test_size=0.2, random_state=25, stratify=labels)
    return train, test

def hm_2_frame(root_path, hm_path, frame_path):
    record_list = [MapRecord(x.strip().split(), root_path) for x in open(hm_path)]

    new_record_list = annotation_attention(record_list)
    str_format = "{} {} {} {}\n"
    with open(frame_path, 'w') as f:
        for record in new_record_list:
            f.write(str_format.format(record.path, record.onset, record.peak, record.label))
    print("Write {} Records to txt file".format(len(new_record_list)))

def hm_2_landmark(root_path, hm_path, frame_path):
    record_list = [MapRecord(x.strip().split(), root_path) for x in open(hm_path)]

    new_record_list = annotation_attention(record_list)
    str_format = "{} {} {} {}\n"
    with open(frame_path, 'w') as f:
        for record in new_record_list:
            re_pa = record.path.replace("_{}","")+".npy"
            f.write(str_format.format(re_pa, record.onset, record.peak, record.label))
    print("Write {} Records to txt file".format(len(new_record_list)))

def hm_2_landmark_v1(record_list, frame_path):
    record_list = record_list

    new_record_list = annotation_attention(record_list)
    str_format = "{} {} {} {}\n"
    with open(frame_path, 'w') as f:
        for record in new_record_list:
            re_pa = record.path.replace("_{}","")+".npy"
            f.write(str_format.format(re_pa, record.onset, record.peak, record.label))
    print("Write {} Records to txt file".format(len(new_record_list)))

In [3]:
def append_record_to_file(record_list, file):
    str_format = "{} {} {} {} {} {} {} {}"
    with open(file, 'a') as f:
        for record in record_list:
            f.write(str_format.format(record.path, record.label, record.onset, record.peak,
                    record.offset, record.width_err, record.height_err, record.index_err)+'\n')
    print("Write {} Records to txt file".format(len(record_list)))

In [ ]:
def generate_ann_file_heatmap(data):
    # data is like subject_folder/emotion_name/index
    pass


def train_test_split(root_path, ann_full_text_file):
    # read full file
    record_list = [MapRecord(x.strip().split(), root_path) for x in open(ann_full_text_file)]



    pass

In [6]:
train_file = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_train_landmark_S5.txt"
train_ld = "C:\\Users\\Zber\\Desktop\\Subjects_Landmark_video\\landmark_train_S5.txt"
hm_2_landmark("", train_file, train_ld)


test_file = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_test_landmark_S5.txt"
test_ld = "C:\\Users\\Zber\\Desktop\\Subjects_Landmark_video\\landmark_test_S5.txt"
hm_2_landmark("", test_file, test_ld)


Write 814 Records to txt file
Write 420 Records to txt file


### Heatmap annotation file to landmark annotation file 


In [ ]:
full_anno_file = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_full_S8.txt"
root_path = ""
record_list = [MapRecord(x.strip().split(), root_path) for x in open(full_anno_file)]
for re in record_list:
    


### Heatmap annotation file to frame annotation file

In [ ]:
from FER.utils import get_label

# loading data configure
annotation_save_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\"
str_arr = []
# str format: path, label, onset, peak, offset, widthError, heightError, indexError
str_format = "{} {} {} {} {} {} {} {}"

emotion_list = ['Joy', 'Surprise', 'Anger', 'Sadness', 'Fear', 'Disgust']
subs = ['W1']
adc_path = "{}_{}"

start_index = 0
end_index = 10
onset = 41
peak = 140

for sub in subs:
    for e in emotion_list:
        for i in range(start_index, end_index):
            relative_path = sub + '/' +adc_path.format(e, i)+ '_{}.npy'
            out = [onset, peak, -1, 0, 0, 0]
            label = get_label(e)
            str_arr.append(str_format.format(relative_path, label, *out))

with open(os.path.join(annotation_save_path, "heatmap_annotation_test_wearmask.txt"), 'a') as f:
    f.writelines('\n'.join(str_arr))
print("Write {} Records to txt file".format(len(str_arr)))

In [18]:
root_path = ""
# new_heatmap = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\annotations_v3.txt"
new_heatmap_ann = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\annotations_S8_new.txt"

train_ann_og = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_new.txt"
train_ann = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_S8.txt"
test_ann = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_S8.txt"

og_train_list = [MapRecord(x.strip().split(), root_path) for x in open(train_ann_og)]
record_list = [MapRecord(x.strip().split(), root_path) for x in open(new_heatmap_ann)]
train_list, test_list = train_test_split(record_list)
train_list = og_train_list + train_list
import random
random.shuffle(train_list)

In [26]:
new_heatmap_ann = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\annotations_S8_new.txt"

train_ann_og = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_new.txt"
test_ann_og = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_new.txt"

og_train_list = [MapRecord(x.strip().split(), root_path) for x in open(train_ann_og)]
og_test_list = [MapRecord(x.strip().split(), root_path) for x in open(test_ann_og)]
ONSET = 31
PEAK = 130
for r in og_train_list:
    r.onset = ONSET
    r.peak = PEAK
for r in og_test_list:
    r.onset = ONSET
    r.peak = PEAK


record_list = [MapRecord(x.strip().split(), root_path) for x in open(new_heatmap_ann)]
train_list, test_list = train_test_split(record_list)
train_list = og_train_list + train_list
import random
random.shuffle(train_list)
test_list = og_test_list + test_list
print(len(train_list))
print(len(test_list))

1288
347


In [27]:
train_ann = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_S8_constant.txt"
test_ann = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_S8_constant.txt"
append_record_to_file(train_list, train_ann)
append_record_to_file(test_list, test_ann)

Write 1288 Records to txt file
Write 347 Records to txt file


In [14]:
subs=['S6','S7']
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"
new_heatmap = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\annotations_v3.txt"
new_heatmap_ann = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\annotations_S8_new.txt"
record_list = [MapRecord(x.strip().split(), "") for x in open(new_heatmap)]
ONSET = 31
PEAK = 130
for r in record_list:
    r.onset = ONSET
    r.peak = PEAK

record_list = annotation_update(record_list)
neutral_list = annotation_append(['S6', 'S7'])

str_format = "{} {} {} {} {} {} {} {}"
# with open(new_heatmap_ann, 'w') as f:
#     for record in record_list:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak,
#                 record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(record_list)))


with open(new_heatmap_ann, 'a') as f:
    for record in neutral_list:
        f.write(record+'\n')
print("Write {} Records to txt file".format(len(neutral_list)))




# neutral_list = annotation_append(['S6', 'S7'])
# print(len(record_list))
# record_list = record_list + neutral_list
# print(len(record_list))
# train, test = data_split(record_list)
# print(len(train))
# print(len(test))

Write 60 Records to txt file


In [62]:
root_path = ""
hm_train = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_S8_v1.txt"
frame_train = "C:\\Users\\Zber\\Desktop\\Subjects_Frames\\video_annotation_train_S8_v1.txt"
hm_2_frame(root_path, hm_train, frame_train)

hm_test = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_S8_v1.txt"
frame_test = "C:\\Users\\Zber\\Desktop\\Subjects_Frames\\video_annotation_test_S8_v1.txt"
hm_2_frame(root_path, hm_test, frame_test)

Write 1216 Records to txt file
Write 333 Records to txt file


In [8]:
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"
hm_train = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_new.txt"
frame_train = "C:\\Users\\Zber\\Desktop\\Subjects_Frames\\video_annotation_train_new.txt"
hm_2_frame(root_path, hm_train, frame_train)

hm_test = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_new.txt"
frame_test = "C:\\Users\\Zber\\Desktop\\Subjects_Frames\\video_annotation_test_new.txt"
hm_2_frame(root_path, hm_test, frame_test)

Write 973 Records to txt file
Write 242 Records to txt file


In [7]:
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"
hm_train = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_new.txt"
landmark_train = "C:\\Users\\Zber\\Desktop\\Subjects_Landmark\\landmark_annotation_train_new.txt"
hm_2_landmark(root_path, hm_train, landmark_train)

hm_test = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_new.txt"
landmark_test = "C:\\Users\\Zber\\Desktop\\Subjects_Landmark\\landmark_annotation_test_new.txt"
hm_2_landmark(root_path, hm_test, landmark_test)

Write 973 Records to txt file
Write 242 Records to txt file


In [ ]:
og_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap_Large\\heatmap_annotation_full_test.txt"
new_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap_Large\\heatmap_annotation_full_test_new.txt"
og = open(og_path, 'r')
new = open(new_path, 'w')

def check_name(name):
    for exp in exp_list:
        if ("S5/"+exp) in name:
            return False
    return True

for l in og:
    if check_name(l):
        new.write(l)
og.close()
new.close()

In [9]:
text_path = "C:\\Users\\Zber\Desktop\\invalid_data.txt"
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_test_Pose.txt"
root_path = ""

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

print(len(record_list))
with open(text_path) as f:
    lines = f.readlines()

count = 0
for l in lines:
    l = l[:-1]
    for index, r in enumerate(record_list):
        if l in r.path:
            print(r.path)
            record_list.pop(index)
            count += 1



# new_annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train_S8_v1.txt"
# append_record_to_file(record_list, new_annotation_path)


168


In [61]:
text_path = "C:\\Users\\Zber\Desktop\\invalid_data_test.txt"
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_S8.txt"
root_path = ""

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

print(len(record_list))
with open(text_path) as f:
    lines = f.readlines()

count = 0
for l in lines:
    l = l[:-1]
    for index, r in enumerate(record_list):
        if l in r.path:
            # print(r.path)
            record_list.pop(index)
            count += 1



new_annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test_S8_v1.txt"
append_record_to_file(record_list, new_annotation_path)

347
Write 333 Records to txt file


In [3]:
heatmap_path_1 = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\S1\\Anger_0_azi.npy"
heatmap_path_2 = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\S6\\Anger_0_azi.npy"
h1 = np.load(heatmap_path_1)
h2 = np.load(heatmap_path_2)
 
h1_mean = np.mean(h1)
h2_mean = np.mean(h2)
print(h1_mean)
print(h2_mean)

73.13816750796464
73.92823863686587


In [43]:
print("iiii".index('i'))

0


# mean and std

In [44]:
arr = []
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

ele = [-np.inf]

for record in record_list:
    
    npy_path = record.path.format('ele')
    d = np.load(npy_path).flatten()
    mask = np.isin(d, ele)
    if len(d[mask]) >0:
        print(npy_path)
    arr.append(d)

arr = np.concatenate(arr, axis=0)

mean = np.mean(arr)
std = np.std(arr)
print("Mean : {}".format(mean))
print("STD : {}".format(std))


Mean : 86.07235512709005
STD : 5.921389444342834


In [45]:
arr = []
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap_Large\\heatmap_annotation_train.txt"
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap_Large"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]
train = []

ele = [-np.inf, np.inf, 0]

for record in record_list:
    
    npy_path = record.path.format('azi')
    d = np.load(npy_path).flatten()
    # mask = d > 1000
    mask = np.isin(d, ele)
    if len(d[mask]) >0:
        print(npy_path)
    # arr.append(d)
    else:
        train.append(record)

# arr = np.concatenate(arr, axis=0)


# str_format = "{} {} {} {} {} {} {} {}"
# with open(os.path.join(root_path, "heatmap_annotation_train.txt"), 'a') as f:
#     for record in train:
#         f.write(str_format.format(record.relative_path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(train)))

In [46]:
root_path = "C:\\Users\\Zber\\Desktop\\Subjects_Frames"
# annotaton_path = "D:\\Subjects\\annotations_v2.txt"
# annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation.txt"
annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
# annotaton_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test.txt"

record_list = [MapRecord(x.strip().split(), root_path) for x in open(annotaton_path)]

# new_record_list = annotation_update(record_list)
#
# # str format: path, label, onset, peak, offset, widthError, heightError, indexError
# str_format = "{} {} {} {} {} {} {} {}"
# with open(os.path.join(root_path, "heatmap_annotation.txt"), 'a') as f:
#     for record in new_record_list:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(new_record_list)))

# train, test = data_split(record_list)

# str format: path, label, onset, peak, offset, widthError, heightError, indexError
# str_format = "{} {} {} {} {} {} {} {}"
# with open(os.path.join(root_path, "heatmap_annotation_train.txt"), 'a') as f:
#     for record in train:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(train)))
#
# with open(os.path.join(root_path, "heatmap_annotation_test.txt"), 'a') as f:
#     for record in test:
#         f.write(str_format.format(record.path, record.label, record.onset, record.peak, record.offset, record.width_err, record.height_err, record.index_err)+'\n')
# print("Write {} Records to txt file".format(len(test)))


# str_arr = annotation_append()
# with open(annotaton_path, 'a') as f:
#     f.writelines('\n'.join(str_arr))
# print("Write {} Records to txt file".format(len(str_arr)))

new_record_list = annotation_attention(record_list)
str_format = "{} {} {} {}\n"
with open(os.path.join(root_path, "annotations_att_train.txt"), 'w') as f:
    for record in new_record_list:
        f.write(str_format.format(record.path, record.onset, record.peak, record.label))
print("Write {} Records to txt file".format(len(new_record_list)))

Write 979 Records to txt file


In [47]:
annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_train.txt"
new_annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_full_train.txt"
root_path = ""
record_list = [MapRecord(x.strip().split(), root_path)
               for x in open(annotation_path)]

str_format = "{} {} {} {} {} {} {} {}"
with open(new_annotation_path, 'w') as f:
    for record in record_list:
        record.onset = 0
        record.peak = 299
        f.write(str_format.format(record.path, record.label, record.onset, record.peak,
                record.offset, record.width_err, record.height_err, record.index_err)+'\n')
print("Write {} Records to txt file".format(len(record_list)))


Write 979 Records to txt file


In [48]:
annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_test.txt"
new_annotation_path = "C:\\Users\\Zber\\Desktop\\Subjects_Heatmap\\heatmap_annotation_full_test.txt"
root_path = ""
record_list = [MapRecord(x.strip().split(), root_path)
               for x in open(annotation_path)]

str_format = "{} {} {} {} {} {} {} {}"
with open(new_annotation_path, 'w') as f:
    for record in record_list:
        record.onset = 0
        record.peak = 299
        f.write(str_format.format(record.path, record.label, record.onset, record.peak,
                record.offset, record.width_err, record.height_err, record.index_err)+'\n')
print("Write {} Records to txt file".format(len(record_list)))

Write 252 Records to txt file


In [ ]:
# copy to subject
source_folder = "D:/Subjects/S0"

# target_folder = "D:/Subjects/S8"
# start_index = 30
# end_index = 60


target_folder = "D:/Subjects/S9"
start_index = 50
end_index = 80

files = []
arr = np.arange(start_index, end_index)
for file in os.listdir(source_folder):
    if ".svo" in file:
        num = file[file.index("_")+1:file.index(".")]
        if int(num) in arr:
            files.append(file)
    else:
        ids = [i for i in range(len(file)) if file[i] == "_"]
        num = file[ids[0]+1:ids[1]]
        if int(num) in arr:
            files.append(file)


import shutil
for file in files:
    src = os.path.join(source_folder, file)
    target_file = file
    if ".svo" in file:
        old_num =target_file[target_file.index("_")+1:target_file.index(".")]
    else:
        ids = [i for i in range(len(file)) if file[i] == "_"]
        old_num =target_file[ids[0]+1:ids[1]]
    new_num = int(old_num) - start_index
    replace_file = target_file.replace(str(old_num), str(new_num)) 
    dst = os.path.join(target_folder, replace_file)
    print("{} --> {}".format(src, dst))
    shutil.copy2(src, dst)



    #     ids = [i for i in range(len(file)) if file[i] == "_"]
    # for id  in range(start_index, end_index):
    #     if len(ids) >= 2:
    #         if file[ids[0]:ids[1]] == id:
    #             print(file)
    #     else:
    #         if file[ids[0]:] == id:
    #             print(file)



In [4]:
import numpy as np
np_path = "C:/Users/Zber/Desktop/Subjects_Landmark/S5/Anger_29.npy"
data = np.load(np_path)

In [3]:
import numpy as np
m = np.asarray([5.789, 10.342, 11.321, 11.931, 13.225, 17.231, 18.932])/100
d = np.asarray([24.597, 17.519, 15.069, 13.572, 11.871, 10.578, 10.101])
std = np.asarray([1.756, 2.060, 3.544, 3.672, 3.863, 4.739, 4.843])/100
(m+std)*d



array([1.85584365, 2.17270638, 2.24000685, 2.11763916, 2.02851648,
       2.3239866 , 2.40151275])